Code copied from https://repo.ops.maap-project.org/gchang/gedi_tutorials/-/blob/main/2_gedi_l4a_subsets.ipynb

From Laura:
- Select only variables of user interest (e.g. RH100, RH50 from L2A, cover and pan from L2B, agbd and agbd_se from L4A)
- Ability to subset by sensitivity attribute to some level (e.g. where sensitivity >0.95)
- Ability to subset using quality flag (L4_quality_flag in L4A, quality_flag in L2A)


In [1]:
import os
from glob import glob
from os import path
import pandas as pd
import geopandas as gpd
import h5py
import numpy as np
from profilehooks import timecall
from typing import List
import multiprocessing

from gedi_utils import get_geo_boundary

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
infile = "GEDI04_A_2019108045816_O01962_01_T01066_02_002_01_V002.h5"
hf_in = h5py.File(infile, 'r')

In [11]:
#@timecall
def subset_gedi_granule(granule: str, aoi, filter_cols: List = ["lat_lowestmode", "lon_lowestmode"]):
    """
    Subset a GEDI granule by a polygon in CRS 4326
    granule = path to a granule h5 file that's already been downloaded
    aoi = a shapely polygon of the aoi
    
    return path to geojson output
    """
    infile = granule

    hf_in = h5py.File(infile, 'r')

    result = subset_h5(hf_in, aoi, filter_cols)
    
    return result

#@timecall
def spatial_filter(beam, aoi):
    
    lat = beam['lat_lowestmode'][:]
    lon = beam['lon_lowestmode'][:]
    i = np.arange(0, len(lat), 1) # index
    geo_arr = list(zip(lat,lon, i))
    l4adf = pd.DataFrame(geo_arr, columns=["lat_lowestmode", "lon_lowestmode", "i"])
    l4agdf = gpd.GeoDataFrame(l4adf, geometry=gpd.points_from_xy(l4adf.lon_lowestmode, l4adf.lat_lowestmode))
    l4agdf.crs = "EPSG:4326"
    # TODO: is it faster with a spatial index, or rough pass with BBOX first?
    bbox = aoi.geometry[0].bounds
    l4agdf_clip = l4agdf.cx[bbox[0]:bbox[2], bbox[1]:bbox[3]]
    l4agdf_gsrm = l4agdf_clip[l4agdf_clip['geometry'].within(aoi.geometry[0])]
    indices = l4agdf_gsrm.i
    
    return indices

#@timecall
def subset_h5(hf_in, aoi, filter_cols):
    subset_df = pd.DataFrame()
    
    # loop through BEAMXXXX groups
    for v in list(hf_in.keys()):
        if v.startswith('BEAM'):
            col_names = []
            col_val = []
            beam = hf_in[v]
            
            indices = spatial_filter(beam, aoi)
            
            # TODO: when to spatial subset?
            for key, value in beam.items():
                # looping through subgroups
                if isinstance(value, h5py.Group):
                    for key2, value2 in value.items():
                        if (key2 not in filter_cols):
                            continue
                        if (key2 != "shot_number"):
                             # xvar variables have 2D
                            if (key2.startswith('xvar')):
                                for r in range(4):
                                    col_names.append(key2 + '_' + str(r+1))
                                    col_val.append(value2[:, r][indices].tolist())
                            else:
                                col_names.append(key2)
                                col_val.append(value2[:][indices].tolist())
                
                #looping through base group
                else:
                    if (key not in filter_cols):
                        continue
                    # xvar variables have 2D
                    if (key.startswith('xvar')):
                        for r in range(4):
                            col_names.append(key + '_' + str(r+1))
                            col_val.append(value[:, r][indices].tolist())
                           
                    else:
                        col_names.append(key)
                        col_val.append(value[:][indices].tolist())
                    
            # create a pandas dataframe        
            beam_df = pd.DataFrame(map(list, zip(*col_val)), columns=col_names) 
            # Inserting BEAM names
            beam_df.insert(0, 'BEAM', np.repeat(str(v), len(beam_df.index)).tolist())
            # Appending to the subset_df dataframe
            subset_df = subset_df.append(beam_df)
    
    hf_in.close()
    # all_gdf = gpd.GeoDataFrame(subset_df, geometry=gpd.points_from_xy(subset_df.lon_lowestmode, subset_df.lat_lowestmode))
    all_gdf = gpd.GeoDataFrame(subset_df.loc[:,~subset_df.columns.isin(['lon_lowestmode', 'lat_lowestmode'])],
                               geometry=gpd.points_from_xy(subset_df.lon_lowestmode, subset_df.lat_lowestmode))
    all_gdf.crs = "EPSG:4326"
    # TODO: Drop the lon and lat columns after geometry creation(or during)
    # TODO: document how many points before and after filtering
    #print(f"All points {all_gdf.shape}")
    subset_gdf = all_gdf[all_gdf['agbd'] >= 0] #Filter out bad? data
    # If supplied apply additional filters
    # l4_quality_flag == 1
    # geolocation/sensitivity_a2 > 0.95
    subset_gdf = all_gdf #Doing the spatial search first didn't help at all, so maybe the spatial query is the slow part.
    print(f"Subset points {subset_gdf.shape}")
    
    return subset_gdf

#@timecall
def write_subset_fgb(infile, gdf):
    """
    Write GeoDataFrame to geojson
    """
    outfile = infile.replace('.h5', '.fgb')
    gdf.to_file(outfile, driver='FlatGeobuf')
    
    return outfile

@timecall
def write_subset_geoparquet(infile, gdf):
    """
    Write GeoDataFrame to geojson
    """
    outfile = infile.replace('.h5', '.geoparquet')
    gdf.to_parquet(outfile)
    
    return outfile

In [ ]:
# Thinking about a parallel version, but will file access lock?
def collect_results(result):
    results.extend(result)

In [4]:
gabon_gdf = get_geo_boundary('GAB', 0)

In [5]:
gabon_gdf.geometry[0].bounds

(8.6954708, -3.978806, 14.502347, 2.322612)

In [5]:
#test = subset_gedi_granule(infile, gabon_gdf)
# Filtered version
filter_cols = ["agbd", "agbd_se", "l4_quality_flag", "sensitivity", "lat_lowestmode", "lon_lowestmode"]
test = subset_gedi_granule(infile, gabon_gdf, filter_cols)

Subset points (64209, 6)


In [6]:
print(test.shape)
test.head()

(64209, 6)


,BEAM,agbd,agbd_se,l4_quality_flag,sensitivity,geometry
0,BEAM0000,-9999.0,-9999.0,0,-0.642939,POINT (10.58764 -3.29189)
1,BEAM0000,-9999.0,-9999.0,0,8.097681,POINT (10.58794 -3.29146)
2,BEAM0000,-9999.0,-9999.0,0,-252.114624,POINT (10.58824 -3.29104)
3,BEAM0000,-9999.0,-9999.0,0,-1.101499,POINT (10.58854 -3.29062)
4,BEAM0000,-9999.0,-9999.0,0,3.414868,POINT (10.58884 -3.29020)


In [27]:
# Reduce to a set of preselect columns
# TODO: it would be better to not even include in the dataframe to start
filter_strings = ["agbd", "agbd_se", "l4_quality_flag", "sensitivity", "geometry"]
cols = list(test.columns)
def Filter(string, substr):
    return [str for str in string if
             any(sub in str for sub in substr)]

final_table_columns = Filter(cols, filter_strings)
#test.drop(columns=test.columns.difference(final_table_columns), inplace=True)
test.drop(columns=test.columns.difference(filter_strings), inplace=True)

In [11]:
write_subset_fgb(infile, test)

'GEDI04_A_2019108045816_O01962_01_T01066_02_002_01_V002.fgb'

In [9]:
# Try to write geoparquet
write_subset_geoparquet(infile, test)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:116: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


'GEDI04_A_2019108045816_O01962_01_T01066_02_002_01_V002.geoparquet'

# Exploration

In [ ]:
# Alternate method using pandas or xarray?
pd_h5 = pd.read_hdf(infile, )

In [16]:
# exploring formats
import fiona 
fiona.supported_drivers

{'ARCGEN': 'r',
 'DXF': 'rw',
 'CSV': 'raw',
 'OpenFileGDB': 'r',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'FlatGeobuf': 'rw',
 'GeoJSON': 'raw',
 'GeoJSONSeq': 'rw',
 'GPKG': 'raw',
 'GML': 'rw',
 'OGR_GMT': 'rw',
 'GPX': 'rw',
 'GPSTrackMaker': 'rw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'OGR_PDS': 'r',
 'S57': 'r',
 'SQLite': 'raw',
 'TopoJSON': 'r'}

In [16]:
#exploring filtering options
filter_strings = ["agbd", "l4_quality_flag", "sensitivity"]
cols = list(test.columns)
def Filter(string, substr):
    return [str for str in string if
             any(sub in str for sub in substr)]
Filter(cols, filter_strings)

['agbd',
 'agbd_pi_lower',
 'agbd_pi_upper',
 'agbd_a1',
 'agbd_a10',
 'agbd_a2',
 'agbd_a3',
 'agbd_a4',
 'agbd_a5',
 'agbd_a6',
 'agbd_pi_lower_a1',
 'agbd_pi_lower_a10',
 'agbd_pi_lower_a2',
 'agbd_pi_lower_a3',
 'agbd_pi_lower_a4',
 'agbd_pi_lower_a5',
 'agbd_pi_lower_a6',
 'agbd_pi_upper_a1',
 'agbd_pi_upper_a10',
 'agbd_pi_upper_a2',
 'agbd_pi_upper_a3',
 'agbd_pi_upper_a4',
 'agbd_pi_upper_a5',
 'agbd_pi_upper_a6',
 'agbd_se_a1',
 'agbd_se_a10',
 'agbd_se_a2',
 'agbd_se_a3',
 'agbd_se_a4',
 'agbd_se_a5',
 'agbd_se_a6',
 'agbd_t_a1',
 'agbd_t_a10',
 'agbd_t_a2',
 'agbd_t_a3',
 'agbd_t_a4',
 'agbd_t_a5',
 'agbd_t_a6',
 'agbd_t_pi_lower_a1',
 'agbd_t_pi_lower_a10',
 'agbd_t_pi_lower_a2',
 'agbd_t_pi_lower_a3',
 'agbd_t_pi_lower_a4',
 'agbd_t_pi_lower_a5',
 'agbd_t_pi_lower_a6',
 'agbd_t_pi_upper_a1',
 'agbd_t_pi_upper_a10',
 'agbd_t_pi_upper_a2',
 'agbd_t_pi_upper_a3',
 'agbd_t_pi_upper_a4',
 'agbd_t_pi_upper_a5',
 'agbd_t_pi_upper_a6',
 'agbd_t_se_a1',
 'agbd_t_se_a10',
 'agbd_t_s

# Bulk Processing

In [13]:
folder_path = '/projects/my-public-bucket/gedi-l4a/gabon'
file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('h5')]
print(len(file_list))

file_list_fgb = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('fgb')]
print(len(file_list_fgb))

file_list_pq = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('geoparquet')]
print(len(file_list_pq))

1009
467
8


In [12]:
# For testing subset the 10 sample granules
for file in file_list[:11]:
    filter_strings = ["agbd", "agbd_se", "l4_quality_flag", "sensitivity", "lat_lowestmode", "lon_lowestmode"]
    subset = subset_gedi_granule(file, gabon_gdf, filter_strings)

    if subset.shape[0] > 0 :
        write_subset_geoparquet(file, subset)
    else:
        outfile = 'False'

Subset points (64209, 6)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:117: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')

  write_subset_geoparquet (/tmp/ipykernel_8553/3431064882.py:111):
    0.457 seconds



Subset points (24866, 6)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:117: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')

  write_subset_geoparquet (/tmp/ipykernel_8553/3431064882.py:111):
    0.340 seconds



Subset points (0, 6)
Subset points (1846, 6)


...

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:117: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')

  write_subset_geoparquet (/tmp/ipykernel_8553/3431064882.py:111):
    0.454 seconds



Subset points (0, 6)
Subset points (0, 6)
Subset points (66053, 6)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:117: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')

  write_subset_geoparquet (/tmp/ipykernel_8553/3431064882.py:111):
    0.418 seconds



Subset points (58430, 6)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:117: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')

  write_subset_geoparquet (/tmp/ipykernel_8553/3431064882.py:111):
    0.445 seconds



## Parallel version

In [11]:
# New variant that does the h5 files in parallel
def subset_pre_granule(file):
    filter_strings = ["agbd", "agbd_se", "l4_quality_flag", "sensitivity", "lat_lowestmode", "lon_lowestmode"]
    subset = subset_gedi_granule(file, gabon_gdf, filter_strings)
    if subset.shape[0] > 0 :
        outfile = write_subset(file, subset)
        print(outfile)
    else:
        outfile = 'False'
        
    return outfile

pool = multiprocessing.Pool(processes=6)
results = pool.map(subset_pre_granule, file_list[9:12])
pool.close()
pool.join()

print(results)
    


Subset points (23275, 6)
/projects/my-public-bucket/gedi-l4a/gabon/GEDI04_A_2019115021551_O02069_02_T03606_02_002_01_V002.fgb
Subset points (58430, 6)
Subset points (66053, 6)
/projects/my-public-bucket/gedi-l4a/gabon/GEDI04_A_2019115021551_O02069_01_T03606_02_002_01_V002.fgb
/projects/my-public-bucket/gedi-l4a/gabon/GEDI04_A_2019114135421_O02061_04_T00905_02_002_01_V002.fgb
['/projects/my-public-bucket/gedi-l4a/gabon/GEDI04_A_2019114135421_O02061_04_T00905_02_002_01_V002.fgb', '/projects/my-public-bucket/gedi-l4a/gabon/GEDI04_A_2019115021551_O02069_01_T03606_02_002_01_V002.fgb', '/projects/my-public-bucket/gedi-l4a/gabon/GEDI04_A_2019115021551_O02069_02_T03606_02_002_01_V002.fgb']


# Misc Exp

In [50]:
for k,v in hf_in["BEAM0000"].items():
    if (k not in filter_cols):
        continue
    else:
        print(k)
        print(v)

agbd
<HDF5 dataset "agbd": shape (63659,), type "<f4">
agbd_se
<HDF5 dataset "agbd_se": shape (63659,), type "<f4">
l4_quality_flag
<HDF5 dataset "l4_quality_flag": shape (63659,), type "|u1">
lat_lowestmode
<HDF5 dataset "lat_lowestmode": shape (63659,), type "<f8">
lon_lowestmode
<HDF5 dataset "lon_lowestmode": shape (63659,), type "<f8">
sensitivity
<HDF5 dataset "sensitivity": shape (63659,), type "<f4">


In [67]:
#df.loc[:, ~df.columns.isin(['rebounds', 'assists'])]
test.loc[:,~test.columns.isin(['lon_lowestmode', 'lat_lowestmode'])].columns

test2 = gpd.GeoDataFrame(test.loc[:,~test.columns.isin(['lon_lowestmode', 'lat_lowestmode', 'geometry'])], 
                         geometry=gpd.points_from_xy(test.lon_lowestmode, test.lat_lowestmode))

In [68]:
test2.head()

,BEAM,agbd,agbd_se,l4_quality_flag,sensitivity,geometry
55637,BEAM0000,-9999.0,-9999.0,0,-0.642939,POINT (10.58764 -3.29189)
55638,BEAM0000,-9999.0,-9999.0,0,8.097681,POINT (10.58794 -3.29146)
55639,BEAM0000,-9999.0,-9999.0,0,-252.114624,POINT (10.58824 -3.29104)
55640,BEAM0000,-9999.0,-9999.0,0,-1.101499,POINT (10.58854 -3.29062)
55641,BEAM0000,-9999.0,-9999.0,0,3.414868,POINT (10.58884 -3.29020)
